## 1.Отчет по реализации модели из статьи <a href=https://arxiv.org/pdf/1808.00158.pdf>SPEAKER RECOGNITION FROM RAW WAVEFORM WITH SINCNET</a>

В данной статье описывалась задача speaker identification (определение человека по куску аудио файла с его речью). 

Особенностью данной статьи является специальный слой нейронной сети (SincNet), которая применяется к исходному аудио файлу.  
Идея SincNet состоит в следующем: использовать не обычную свертку, а фильтр следующего вида  
$$G(f, f_1, f_2)=rect(\frac{f}{2f_2})-rect(\frac{f}{2f_1})$$
где $rect(f)$ - прямоугольная функция. 
![рис. 1](images/Rectangular_function.svg.png)

Используя преобразование Фурье для этой функции, мы можем получить ее аналог во временной области:
$$g(n, f_1, f_2)=2f_2sinc(2\pi f_2 n)-2f_1sinc(2\pi f_1 n)$$
    где $sinc(t)=\frac{sin(t)}{t}$. $f_1$ и $f_2$ - это соответственно обучаемые наименьшая и наибольшая частота среза (таким образом, нам нужно обучать $2*s$ параметров, где $s$ - количество фильтров, а не $k*s$, где $k$ - размер ядра в одномерной свертке). В статье предлагается инициализировать $f_1$ и $f_2$ двумя способами, я выбрал вариант с равномерной инициализацией из отрезка $[0, f_s/2]$, где $f_s$ - частота дискретизации. Так как функция $g(t)$ - это идеальный полосовой фильтр, то он требует бесконечное число элементов. Для его аппроксимации с конечным числом элементов применяется метод окон. В данном случае используется окно Хэмминга:
$$g_w[n,f_1,f_2]=g[n,f_1,f_2]*w[n]$$
$$w[n]=0.54-0.46*cos(\frac{2\pi n}{L})$$


Сама архитектура сети имеет следующий вид: сначала идет слой SincNet, дальше идут обычные сверточные слои с max-пулингом и нормализацией(в данном блоке используется только Layer normalization). Далее идет блок с полносвязными слоями, батч-нормализацией. Функция активации везде - Leaky ReLU. Веса в полносвязном блоке предлагается инициализировать инициализацией Xavier. 
![рис. 1](images/SincNet.png)

Параметры и архитектура сети(параметры были выбраны аналогично параметрам, написанным в статье):

In [1]:
from SincNet import SincNet
SincNet()

SincNet(
  (conv_layers): Sequential(
    (0): LayerNorm((1, 3200), eps=1e-05, elementwise_affine=True)
    (1): ConvSinc()
    (2): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
    (3): LayerNorm((80, 983), eps=1e-05, elementwise_affine=True)
    (4): LeakyReLU(negative_slope=0.2)
    (5): Conv1d(80, 60, kernel_size=(5,), stride=(1,))
    (6): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
    (7): LayerNorm((60, 326), eps=1e-05, elementwise_affine=True)
    (8): LeakyReLU(negative_slope=0.2)
    (9): Conv1d(60, 60, kernel_size=(5,), stride=(1,))
    (10): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
    (11): LayerNorm((60, 107), eps=1e-05, elementwise_affine=True)
    (12): LeakyReLU(negative_slope=0.2)
  )
  (linear_layers): Sequential(
    (0): LayerNorm((6420,), eps=1e-05, elementwise_affine=True)
    (1): Linear(in_features=6420, out_features=2048, bias=True)
    (2): BatchNorm1d(2048, eps=1e

Для проведения эксперимента был выбран датасет TIMIT (так как результаты в статье в основном были приведены для этого датасета, данные уже были разделены на train и test, а так же размер данных был довольно маленький (порядка 1 гб в сжатом виде)), который сначала обрабатывается (в исходных данных удаляются начальная и конечная паузы, а амплитуда каждого речевого высказывания нормализуется) (код для пред обработки был взят из исходного репозитория статьи). Результаты приведены на следующих графиках:

   .                         |.
:-------------------------:|:-------------------------:
![](images/err_snt.png)  |  ![](images/err.png)

Как мы видим, результат получился немного хуже, чем он представлен в статье, хотя одна из особенностей данной нейросети (скорость обучения и быстрая сходимость) была сохранена. Разница в результатах может быть связана с другой инициализацией частот в Sinc-слое, а так же возможными правками или доработками, которые авторы статьи вносили в код, но уже после написания самой статьи.

Как уже было сказано ранее, решалась задача определения человека по голосу (в данном случае была задача классификации с 462 классами), но в статье так же предлагается метод не решения задачи классификации, а нахождения вектора человека по голосу (вместо слоя softmax на конце выходом нейронной сети будет d-мерный вектор, который должен идентифицировать определенного человека). Кроме того, данную нейросеть можно применять к задаче верификации человека по голосу. Соответственно, это направление может быть довольно перспективным.

## 2. Статья за последний год

За последний год мне приглянулось статья <a href=https://arxiv.org/pdf/1706.03762.pdf>Attention Is All You Need</a>. Я хотел лучше разобраться в работе трансформеров, с чем данная статья мне и помогла. Возможно использовать архитектуры трансформеров (скорее всего это или реализуется или уже реализовано) совместо с моделями генерации голосовых сообщений для создания/улучшения голосовых помощников.